In [1]:
import pandas as pd
import plotly.express as px

In [2]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['edition','result_id','athlete','athlete_id','pos'])


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1



In [3]:
athlete_results.drop(repeat, inplace=True)

In [4]:
athlete_results.drop(columns=['edition_id', 'isTeamSport'], inplace=True)

In [5]:
yn_medal = []
for medal in athlete_results['medal']:
    if medal == None:
        yn_medal.append("No medal")
    else:
        yn_medal.append("With medal")
athlete_results['with_medal'] = yn_medal

In [6]:
country = 'POR'
gr = pd.DataFrame(athlete_results.groupby(['country_noc', 'sport', 'with_medal']).count()).reset_index()

gr.rename(columns={"event": "count"}, inplace=True)

new_gr = gr[gr['country_noc'] == country]

display(new_gr)

,country_noc,sport,with_medal,index,count,medal
3913,POR,Archery,No medal,8,8,0
3914,POR,Art Competitions,No medal,15,15,0
3915,POR,Artistic Gymnastics,No medal,145,145,0
3916,POR,Athletics,No medal,341,341,0
3917,POR,Athletics,With medal,12,12,12
3918,POR,Badminton,No medal,11,11,0
3919,POR,Beach Volleyball,No medal,4,4,0
3920,POR,Boxing,No medal,1,1,0
3921,POR,Canoe Slalom,No medal,5,5,0
3922,POR,Canoe Sprint,No medal,36,36,0


In [7]:
fig = px.sunburst(new_gr, path=['sport', 'with_medal'], values='count')
fig.show()

In [28]:
athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)


athlete_results = athlete_Event_Results[athlete_Event_Results["edition"].str.contains("Winter") == False]   # apenas jogos de verao

athlete_results = athlete_results.reset_index()

athlete_results = athlete_results.drop(columns=['result_id','athlete','athlete_id','pos', 'index'])

athlete_results['edition'] = athlete_results['edition'].str.split().str[0].astype(int)


repeat = []
i=0
while(i < len(athlete_results)):
    if athlete_results.loc[i, 'isTeamSport'] == True:
        auxi = i + 1 
        p = athlete_results.loc[i, 'country_noc']
        e = athlete_results.loc[i, 'event']
        a = athlete_results.loc[i, 'edition_id']
        while((athlete_results.loc[auxi, 'country_noc'] == p) and (athlete_results.loc[auxi, 'event'] == e) and (athlete_results.loc[auxi, 'edition_id'] == a)):
            repeat.append(auxi)
            auxi += 1
        i = auxi-1
    i+=1

athlete_results.drop(repeat, inplace=True)

athlete_results['medal'] = athlete_results['medal'].replace([None], 0)
athlete_results['medal'] = athlete_results['medal'].replace(['Gold'], 1)
athlete_results['medal'] = athlete_results['medal'].replace(['Silver'], 1)
athlete_results['medal'] = athlete_results['medal'].replace(['Bronze'], 1)

athlete_results = athlete_results.drop(columns=['edition_id','sport','event','isTeamSport'])

display(athlete_results)


,edition,country_noc,medal
0,1908,ANZ,0
1,1908,ANZ,0
2,1908,ANZ,0
3,1908,ANZ,0
4,1908,ANZ,0
...,...,...,...
253177,2020,USA,0
253178,2020,USA,1
253179,2020,USA,0
253180,2020,USA,1


In [65]:
country = "POR"

country_df = athlete_results[athlete_results["country_noc"] == country]

medals = pd.DataFrame(country_df.groupby(['edition']).sum()).reset_index()

years = country_df["edition"].unique()

year = []
medal = []
count = []

for y in years:
    year.append(y)
    year.append(y)

    medal.append("Without")
    medal.append("With")

    withm = len(country_df[(country_df['edition'] == y) & (country_df['medal'] == 1)])
    withoutm = len(country_df[(country_df['edition'] == y) & (country_df['medal'] == 0)])

    count.append(withoutm)
    count.append(withm)

df = pd.DataFrame()
df["year"] = year
df["medal"] = medal
df["count"] = count

display(df)


,year,medal,count
0,1912,Without,11
1,1912,With,0
2,1920,Without,14
3,1920,With,0
4,1924,Without,36
5,1924,With,1
6,1928,Without,23
7,1928,With,1
8,1932,Without,9
9,1932,With,0


In [66]:
fig = px.bar(df, x="count", y="year", color="medal", title="<b>Medals vs Participations", width=400, height=800, orientation='h')

fig.update_layout(font_family= 'Cabin',autosize = False, 
                      legend=dict(yanchor="top", xanchor="left", font=dict(size=15)),
                      margin=dict(l=1, r=2, b=20, t=31, pad=0))


fig.show()

In [14]:
import plotly.express as px

display(athlete_results)

long_df = px.data.medals_long()

display(long_df)

fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()

,index,country_noc,sport,event,medal,with_medal
0,0,ANZ,Athletics,"100 metres, Men",None,No medal
1,1,ANZ,Athletics,"400 metres, Men",None,No medal
2,2,ANZ,Athletics,"800 metres, Men",None,No medal
3,3,ANZ,Athletics,"800 metres, Men",None,No medal
4,4,ANZ,Athletics,"800 metres, Men",None,No medal
...,...,...,...,...,...,...
253177,314491,USA,Wrestling,"Featherweight, Freestyle, Women",None,No medal
253178,314492,USA,Wrestling,"Lightweight, Freestyle, Women",Bronze,With medal
253179,314493,USA,Wrestling,"Middleweight, Freestyle, Women",None,No medal
253180,314494,USA,Wrestling,"Light-Heavyweight, Freestyle, Women",Gold,With medal


,nation,medal,count
0,South Korea,gold,24
1,China,gold,10
2,Canada,gold,9
3,South Korea,silver,13
4,China,silver,15
5,Canada,silver,12
6,South Korea,bronze,11
7,China,bronze,8
8,Canada,bronze,12


In [8]:
athlete_Bio = pd.read_csv('Dataset/Olympic_Athlete_Bio.csv', sep=',')
athlete_Bio = athlete_Bio.replace(["na"], None)
athlete_Bio["height"] = athlete_Bio["height"].astype(float)

athlete_Event_Results = pd.read_csv('Dataset/Olympic_Athlete_Event_Results.csv', sep=',')
athlete_Event_Results = athlete_Event_Results.replace(["na"], None)

In [9]:
a_bio = {'athelete_id': athlete_Bio['athlete_id'], 'sex': athlete_Bio['sex']}
a_event = {'athelete_id': athlete_Event_Results['athlete_id'], 'medal': athlete_Event_Results['medal']}

df = pd.merge(pd.DataFrame(a_bio), pd.DataFrame(a_event), on='athelete_id')

count = pd.DataFrame(df.groupby(['sex']).count())
count = count['athelete_id']

df1 = df.dropna(subset=['medal'])

print(count)

perc = pd.DataFrame(df1.groupby(['sex', 'medal']).count()).reset_index()


perc.loc[perc["sex"] == "Male", "athelete_id"] = (perc['athelete_id']/count[1])*100
perc.loc[perc["sex"] == "Female", "athelete_id"] = (perc['athelete_id']/count[0])*100

perc.rename(columns={"athelete_id": "count"}, inplace=True)
print(perc)

fig = px.sunburst(perc, path=['medal', 'sex'], values='count', color_discrete_map={'Gold': '#D6AF36', 'Silver': '#A7A7AD', 'Bronze': '#A77044'})
fig.show()


sex
Female     89310
Male      225597
Name: athelete_id, dtype: int64
      sex   medal     count
0  Female  Bronze  5.040869
1  Female    Gold  5.002799
2  Female  Silver  4.969208
3    Male  Bronze  4.610877
4    Male    Gold  4.680470
5    Male  Silver  4.522223


In [10]:
a_bio = {'athelete_id': athlete_Bio['athlete_id'], 'sex': athlete_Bio['sex']}
a_event = {'athelete_id': athlete_Event_Results['athlete_id'], 'edition': athlete_Event_Results['edition']}
a_event['edition'] = a_event['edition'].str.split(' ')

a_event = {'athelete_id': a_event['athelete_id'], 'year': a_event['edition'].str[0], 'season': a_event['edition'].str[1]}

df = pd.merge(pd.DataFrame(a_bio), pd.DataFrame(a_event), on='athelete_id')

count = pd.DataFrame(df.groupby(['year', 'sex', 'season']).count()).reset_index()
count.rename(columns={"athelete_id": "count"}, inplace=True)

print(count)

summer = count[count['season'] == 'Summer']
winter = count[count['season'] == 'Winter']

fig = px.sunburst(summer, path=['year', 'sex'], values='count')
fig.show()

fig = px.sunburst(winter, path=['year', 'sex'], values='count')
fig.show()

     year     sex  season  count
0    1896  Female  Summer      2
1    1896    Male  Summer    617
2    1900  Female  Summer     53
3    1900    Male  Summer   3094
4    1904  Female  Summer     17
..    ...     ...     ...    ...
104  2018    Male  Winter   3036
105  2020  Female  Summer   7126
106  2020    Male  Summer   7660
107  2022  Female  Winter   2430
108  2022    Male  Winter   2958

[109 rows x 4 columns]
